In [ ]:
# PyTorch 라이브러리 import  *** 해당 cell을 수정하지 말 것 ***
import torch
import torch.nn as nn
from torchvision import transforms, datasets

In [ ]:
# TODO: CIFAR-100 training set 불러오기


batch_size=16
train_dataset = datasets.CIFAR100(root='.', train=False, download=True, transform=transforms.ToTensor())

train_size = int(len(train_dataset) * 0.8)
val_size = len(train_dataset) - train_size

train_set, val_set = torch.utils.data.random_split(train_dataset, [train_size, val_size])

train_loader = torch.utils.data.DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset=val_set, batch_size=batch_size, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available else "cpu"))

In [ ]:
class ResBlock(nn.Module):
  def __init__(self):
    super(ResBlock, self).__init__()
    torch.manual_seed(2024)

    self.conv1 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, stride=1, padding=1)
    self.bn1 = nn.BatchNorm2d(num_features=16)

    self.conv2 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, stride=1, padding=1)
    self.bn2 = nn.BatchNorm2d(num_features=16)

    self.activation = nn.ReLU()

  def forward(self, x: torch.tensor):
    out = self.activation(self.bn1(self.conv1(x)))
    out = self.bn2(self.conv2(out))

    x = self.activation(x + out)

    return x

class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    torch.manual_seed(2024) # 결과 재현을 위한 seed number 고정 *** 해당 line을 수정하지 말 것 ***
    # TODO: CNN 구성 layer들 선언

    self.cnn = nn.Sequential(
      nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1),
      nn.BatchNorm2d(num_features=16),
      nn.MaxPool2d(kernel_size=2),
    )

    self.layer1 = ResBlock()
    self.layer2 = ResBlock()
    self.layer3 = ResBlock()
    self.layer4 = ResBlock()
    self.layer5 = ResBlock()
    self.layer6 = ResBlock()
    self.layer7 = ResBlock()

    self.conv4 = nn.Conv2d(in_channels=16, out_channels=256, kernel_size=3, stride=1, padding=1)
    self.flatten = nn.Flatten()

    self.affine = nn.Sequential(
      nn.Linear(65536, 2048),
      nn.ReLU(inplace=True),
      nn.BatchNorm1d(2048),
      nn.Dropout(0.2),

      nn.Linear(2048, 512),
      nn.ReLU(inplace=True),
      nn.BatchNorm1d(512),
      nn.Dropout(0.2),

      nn.Linear(512, 128),
      nn.ReLU(inplace=True),
      nn.BatchNorm1d(128),
      nn.Dropout(0.2),

      nn.Linear(128, 100)
    )

  def forward(self, x):
    # TODO: forward pass 정의

    x = self.cnn(x)
    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x)
    x = self.layer5(x)
    x = self.layer6(x)
    x = self.layer7(x)

    x = self.conv4(x)
    x = self.flatten(x)

    x = self.affine(x)

    return x

model = CNN().to(device)

In [ ]:
from tqdm.notebook import tqdm
# TODO: model 명의로 생성된 CNN 모델에 대해 학습 수행하기

lr = 1e-3
epochs = 50

optimizer = torch.optim.RMSprop(model.parameters(), lr)
loss_fn = torch.nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)

for epoch in tqdm(range(epochs), "epochs", leave=True):
    model.train()
    for image, label in tqdm(train_loader, "batch", leave=False):
        image, label = image.to(device), label.to(device)
        output = model(image)

        loss = loss_fn(output, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    scheduler.step()

    num_correct = 0
    model.eval()

    with torch.no_grad():
        for image, label in tqdm(val_loader, "val", leave=False):
            image, label = image.to(device), label.to(device)

            output = model(image)

            pred = output.argmax(dim=1)
            num_correct += (pred == label).sum()

    print(f"Epoch {epoch + 1} / loss: {scheduler.get_last_lr()[0]:.6e} / Acc: {num_correct / len(val_set) * 100:.2f}%")

In [ ]:
# 학습된 모델 평가  *** 해당 cell을 수정하지 말 것 ***


test_dataset = datasets.CIFAR100(root='.', train=False, download=True,
                                 transform=transforms.ToTensor())
batchsize = 64
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=batchsize, shuffle=True)

num_correct = 0

model.eval()

with torch.no_grad():
  for image, label in test_loader:
    image, label = image.to(device), label.to(device)
    output = model(image)
    pred = output.argmax(dim=1)
    num_correct += (pred == label).sum()

print(f'Accuracy : {num_correct / len(test_dataset) * 100:.2f} %')